# Day 20

截止day 21时最让我困惑的一题，特别是第二部分，对题目的理解产生偏差，无法完全理解意图，导致消耗了大量时间来调试程序。因此首先完成了day 21的两个部分才回来重新考虑Part II。总算读懂了题目之后，又犯了一个低级错误，由于很难写出合适的单元测试，因此耗费了大量时间。

## Part I

整个问题与矩阵相关，因此无法不让我直接用上Numpy库，而且这里的图像旋转、翻转等操作也是Numpy内置的函数，不光应用简单，性能还非常优秀，所以这里就直接上Numpy了。在Rust当中就没有这么好的事情了，这些操作都得自己写代码完成。

第一部分是矩阵的边对齐，可以直接使用Numpy的矢量相等来判断，不过这里采用了一个更加高效的方法，因此观察到第一部分并不需要实际的对矩阵进行旋转和翻转操作，因此实际上只需要对比拼图的四个边即可，因为图像的像素是二元的，所以，可以更进一步将一条边想象成一个二进制数，所以实际上只需要四个无符号整数即可代表一个子图像。

再考虑空间换时间，因为此时一个子图像的四条边只需要表达成四个无符号整数，所以，我们可以使用8个四元组即可表示图像的全部可能性，包括原图、顺时针旋转90°、180°、270°、垂直翻转、水平翻转、延主对角线翻转（矩阵转置）、延次对角线翻转一共八种可能。在创建子图像的时候即可预先计算得到。例如`'#....#..##'`可以表示成二进制的1000010011，当旋转或翻转时，顺序可能会相反，变为`'##..#....#'`，也就是二进制的1100100001，四条边八个二进制数经过组合之后，即可得到8个四元组，完全表示子图像的四条边的八种情况。

下面定义的Tile类即完成这个逻辑：

In [1]:
from typing import Tuple, List
import numpy as np

def array_transform_funcs() -> List[callable]:
    '''
    定义所有的矩阵操作函数，共八个
    '''
    return [
        lambda x: x, # 原矩阵
        lambda x: np.rot90(x, -1), # 顺时针旋转90
        lambda x: np.rot90(x, 2), # 顺时针旋转180
        lambda x: np.rot90(x, 1), # 顺时针旋转270
        lambda x: x[::-1, :], # 垂直翻转
        lambda x: x[:, ::-1], # 水平翻转
        lambda x: np.rot90(np.rot90(x).T, -1), # 次对角线翻转
        lambda x: x.T, # 主对角线翻转
            ]

class Tile(object):
    def __init__(self, data: str):
        '''
        将输入中的一块数据完全转换为一个Tile对象
        '''
        lines = data.split('\n')
        
        # 记录ID值
        self.tile_id = int(lines[0].split(' ')[1].rstrip(':'))
        
        # 从第二行开始后面的是子图像的像素内容
        lines = [line for line in lines[1:] if line]
        # 转换二进制的函数
        translate = lambda x: '0' if x == '.' else '1'
        # 顶部行的二进制转换
        top = ''.join(translate(x) for x in lines[0])
        top_rev = int(top[::-1], 2) # 反向的二进制
        top = int(top, 2) # 正向的二进制
        
        # 左边列的二进制转换，下同
        left = ''.join(translate(line[0]) for line in lines)
        left_rev = int(left[::-1], 2)
        left = int(left, 2)
        
        right = ''.join(translate(line[-1]) for line in lines)
        right_rev = int(right[::-1], 2)
        right = int(right, 2)
        
        bottom = ''.join(translate(x) for x in lines[-1])
        bottom_rev = int(bottom[::-1], 2)
        bottom = int(bottom, 2)
        
        # 子图像四条边的所有可能组合情况，8个四元组，原图、顺时针90、180、270、垂直翻转、水平翻转、主对角线翻转、次对角线翻转
        self.all_forms = ((top, right, bottom, left), (left_rev, top, right_rev, bottom),
                          (bottom_rev, left_rev, top_rev, right_rev), (right, bottom_rev, left, top_rev),
                          (bottom, right_rev, top, left_rev), (top_rev, left, bottom_rev, right),
                          (right_rev, top_rev, left_rev, bottom_rev), (left, bottom, right, top))
        
        # 第二部分需要的图像矩阵数据，除去四条边外的所有像素数据
        arr = np.array([list(line[1:-1]) for line in lines[1:-1]])
        self.arr = (arr == '#').astype('i1')
        
    def set_form(self, form: Tuple[int, int, int, int]):
        '''
        用于记录子图像能够拼接成原始大图的四条边情况，只需要记录一个四元组即可按照
        算法还原子图像的变化情况
        '''
        self.form = form
        
    def get_array_form(self) -> np.ndarray:
        '''
        获得能够拼成原始大图的子图像的矩阵形式
        '''
        form_index = self.all_forms.index(self.form)
        if not 0 <= form_index < len(self.all_forms):
            raise ValueError('form not exists')
        return array_transform_funcs()[form_index](self.arr)

下面定义一个类来表示原始大图，对于第一部分而言，我们需要使用递归的方式去尝试所有可能的组合，由于我们前面已经将所有子图像的边的可能性都表示成了8个四元组，本来比较耗时的计算被优化了：

In [2]:
from typing import List

class BigImage(object):
    def __init__(self, tiles: List[Tile]):
        self.shape = int(len(tiles) ** .5) # 原始大图应该是 n * n 个小图像组成
        self.tiles = tiles # 组成大图的所有可能的小拼图
        self.result = [] # 记录最终拼图结果的列表
        self.image = None # 第二部分要用到的拼接后的图像矩阵
    
    def fits(self, row: int, col: int) -> bool:
        '''
        拼接图像逻辑的关键方法，使用递归累进去试每一张小图像，能否对接左边和上边
        '''
        # 试验下一个拼图，尝试所有未在结果中暂存的小图像
        for tile in self.tiles:
            if tile in self.result:
                continue
            # 先将小图像放入结果列表
            self.result.append(tile)
            # 计算上方小图像的列表序号和左边小图像的列表序号，没有则为-1
            top_index = (row - 1) * self.shape + col if row > 0 else -1
            left_index = row * self.shape + col - 1 if col > 0 else -1
            
            # 尝试当前小图像的所有8中边的可能组合
            for form in tile.all_forms:
                tile.set_form(form) # 记录当前的四条边的组合情况
                
                # 如果当前子图像的可能四元组的左边等于左边子图像（存在的话）的右边
                # 并且当前子图像的可能四元组的顶边等于上方子图像（存在的话）的底边
                # 则找到一个满足的组合
                if (((top_index >= 0 and form[0] == self.result[top_index].form[2]) or top_index < 0) 
                    and ((left_index >= 0 and form[3] == self.result[left_index].form[1]) or left_index < 0)):
                    
                    # 如果已经是最后一个子图像，返回True
                    if row == self.shape - 1 and col == self.shape - 1:
                        return True
                    # 计算下一个子图像在大图中行和列的序号
                    new_col = col + 1 if col < self.shape - 1 else 0
                    new_row = row + 1 if new_col == 0 else row
                    
                    # 递归计算一个子图像，如果返回True，直接返回True
                    if self.fits(new_row, new_col):
                        return True
            # 所有子图像的元组都尝试完，没有找到合适的，将当前子图像从结果列表中pop出来，继续尝试下一个子图像
            self.result.pop()
        # 所有子图像都尝试完了，返回False
        return False
    
    def create_whole_image(self):
        '''
        第二部分需要用的方法，将拼接好的大图去掉子图像的边后组合成一个 n * n 的图像矩阵，用来搜索海怪
        '''
        if not self.result:
            self.fits(0, 0)
        # 最终矩阵的边长，子图像数组边长和大图边长的乘积
        shape = self.result[0].arr.shape[0] * self.shape
        # 初始化一个没有行的大图矩阵用来垂直堆叠每一行的大矩阵
        arr = np.empty((0, shape), dtype='i1')
        for row in range(self.shape):
            # 初始化一个没有列的小图矩阵用来垂直堆叠每一个小图像矩阵
            row_array = np.empty((self.result[0].arr.shape[0], 0), dtype='i1')
            for col in range(self.shape):
                index = row * self.shape + col
                # 堆叠行，利用get_array_form方法获得符合拼接的矩阵
                row_array = np.hstack([row_array, self.result[index].get_array_form()])
            # 堆叠列
            arr = np.vstack([arr, row_array])
        self.image = arr

下面是处理输入的函数，很简单，将文件分成每一块，传递给Tile类构造器即可：

In [3]:
def read_input(input_file: str) -> BigImage:
    with open(input_file) as fn:
        parts = fn.read().split('\n\n')
    tiles = [Tile(p) for p in parts if p]
    return BigImage(tiles)

第一部分的求值逻辑，使用fits来拼图，拼图成功后，将四个角的子图像ID相乘即可：

In [4]:
def part1_solution(image: BigImage) -> int:
    if image.fits(0, 0):
        tile_ids = [r.tile_id for r in image.result]
        return (tile_ids[0] * tile_ids[image.shape - 1] * 
                tile_ids[image.shape * (image.shape - 1)] * tile_ids[-1])
    return -1

单元测试：

In [5]:
test_image = read_input('testcase1.txt')
assert(part1_solution(test_image) == 20899048083289)

下面就是第一部分的结果：

In [6]:
big_image = read_input('input.txt')
part1_solution(big_image)

174206308298779

## Part II

第二部分的坑很多，首先就是题目的每句话都要读清楚，注意原始大图是由小图像去掉边之后拼接而成的，没看到这句话，让我检查了接近两个小时。Tile类中已经定义了get_array_form方法，BigImage类中也定义了create_whole_image方法，这两个方法用来拼接原始的大图像，注释在前面的代码中已经写了，可以回头再次查阅。

第二个坑是再次旋转和翻转的是拼接后的大图像，而不是小图像，这点需要认真再次看题才能看到。

然后其实就是采用滑动窗口在大图的各种旋转和翻转中去找海怪的模式，这里其实不难，只要了解滑动窗口应该都能写出来。当然，使用了Numpy之后，很多代码变得非常简单，例如对比窗口和海怪图案，只需要使用二进制与，然后再与海怪图案做相等比较即可，甚至可以直接用运算符完成。

下面首先获取海怪的图案模式：

In [7]:
def monster_pattern(data: str) -> np.ndarray:
    arr = np.array([list(line) for line in data.split('\n') if line])
    return (arr == '#').astype('i1')

monster = monster_pattern('                  # \n#    ##    ##    ###\n #  #  #  #  #  #   ')
monster

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]],
      dtype=int8)

下面使用滑动窗口法，在原始大图中找到所有海怪图案的数量，如果找到，返回滑动窗口左上角的坐标，否则返回空列表：

In [8]:
def find_monster(image: np.ndarray, monster: np.ndarray) -> List[Tuple[int, int]]:
    positions = []
    
    # 两个循环用来滑动窗口移动
    for x in range(image.shape[1] - monster.shape[1] + 1):
        for y in range(image.shape[0] - monster.shape[0] + 1):
            image_slice = image[y:y+monster.shape[0], x:x+monster.shape[1]]
            diff = (image_slice & monster).astype('i1') # 窗口与海怪图案二进制与
            # 如果结果和海怪图案相等，表示找到，将坐标添加到结果中
            if np.all(diff == monster):
                positions.append((y, x))
                
    return positions

最后就是实现第二部分求值的逻辑，仍然是8个图像处理函数，遍历一遍，如果找到海怪，只需要将海怪区域的图像与海怪图案进行按元素减法，最后整个矩阵将会去除所有的海怪图案，只需要使用Numpy的sum计算矩阵总和，即为结果：

In [9]:
def part2_solution(image: BigImage, monster:np.ndarray) -> int:
    image.create_whole_image() # 拼接原始大图像
    for func in array_transform_funcs(): # 遍历八种不同的旋转、翻转组合
        # 使用find_monster函数搜索海怪位置
        pos = find_monster(func(image.image), monster)
        if pos:
            sea_image = image.image.copy() # 只是为了不改变原始图像内容，可以不需要
            for p in pos:
                # 将海怪区域中的图像值减去海怪图案，即抹去海怪图案像素
                sea_image[p[0]:p[0]+monster.shape[0], p[1]:p[1]+monster.shape[1]] -= monster
            return np.sum(sea_image)

单元测试：

In [10]:
assert(part2_solution(test_image, monster) == 273)

最后就是第二部分的结果：

In [11]:
part2_solution(big_image, monster)

2409